In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
import acquire
import prepare

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
rawdf = acquire.get_zillow_data()

In [4]:
rawdf.shape

(52441, 11)

In [5]:
rawdf.head()

,longitude,latitude,bedrooms,bathrooms,square_feet,lot_size,has_pool,fips_code,age,assessed_value,tax_amount
0,"-118,740133","34,251502",4.00,2.00,"1,323.00","6,825.00",NaN,06111,53.00,"453,000.00","5,235.32"
1,"-118,217048","33,798657",4.00,2.00,"1,339.00","6,107.00",NaN,06037,40.00,"298,371.00","3,714.58"
2,"-118,531253","34,039467",2.00,2.00,"1,231.00","11,837.00",NaN,06037,67.00,"161,897.00","2,031.39"
3,"-117,613897","33,663005",5.00,3.00,"3,008.00","6,760.00",1.00,06059,22.00,"472,384.00","7,234.74"
4,"-118,393309","34,227422",2.00,1.00,996.00,"7,518.00",NaN,06037,77.00,"58,693.00",751.66


In [6]:
df = prepare.prep_zillow(rawdf)

In [7]:
df.shape

(45324, 23)

In [8]:
df.head()

,longitude,latitude,bedrooms,bathrooms,square_feet,lot_size,has_pool,fips_code,age,assessed_value,...,bathrooms_size,county_Los Angeles,county_Orange,county_Ventura,bedrooms_size_small,bedrooms_size_medium,bedrooms_size_large,bathrooms_size_small,bathrooms_size_medium,bathrooms_size_large
0,"-118,740133","34,251502",4.00,2.00,"1,323.00","6,825.00",0.00,06111,53,"453,000.00",...,small,0,0,1,0,1,0,1,0,0
1,"-118,217048","33,798657",4.00,2.00,"1,339.00","6,107.00",0.00,06037,40,"298,371.00",...,small,1,0,0,0,1,0,1,0,0
2,"-118,531253","34,039467",2.00,2.00,"1,231.00","11,837.00",0.00,06037,67,"161,897.00",...,small,1,0,0,1,0,0,1,0,0
3,"-117,613897","33,663005",5.00,3.00,"3,008.00","6,760.00",1.00,06059,22,"472,384.00",...,medium,0,1,0,0,0,1,0,1,0
4,"-118,393309","34,227422",2.00,1.00,996.00,"7,518.00",0.00,06037,77,"58,693.00",...,small,1,0,0,1,0,0,1,0,0


## Simple Model
$$ y = mx + b $$

$$y = f(x)$$

Tested out all scalers, and MinMaxScaler performs the best

In [9]:
scaled_df = df.copy()

In [10]:
scaler = MinMaxScaler()

In [ ]:
# scaler = sklearn.preprocessing.RobustScaler()

In [ ]:
# # Quantile scaler
# qt = sklearn.preprocessing.QuantileTransformer(n_quantiles=10, output_distribution='normal', random_state=0)

In [ ]:
# qt.fit(scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']])

In [ ]:
# scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']] = qt.transform(scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']])

In [11]:
scaler.fit(scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']])

MinMaxScaler()

In [12]:
scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']] = scaler.transform(scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']])

In [13]:
scaled_df.head()

,longitude,latitude,bedrooms,bathrooms,square_feet,lot_size,has_pool,fips_code,age,assessed_value,...,bathrooms_size,county_Los Angeles,county_Orange,county_Ventura,bedrooms_size_small,bedrooms_size_medium,bedrooms_size_large,bathrooms_size_small,bathrooms_size_medium,bathrooms_size_large
0,"-118,740133","34,251502",0.60,0.20,0.19,0.06,0.00,06111,0.38,"453,000.00",...,small,0,0,1,0,1,0,1,0,0
1,"-118,217048","33,798657",0.60,0.20,0.20,0.05,0.00,06037,0.28,"298,371.00",...,small,1,0,0,0,1,0,1,0,0
2,"-118,531253","34,039467",0.20,0.20,0.17,0.11,0.00,06037,0.48,"161,897.00",...,small,1,0,0,1,0,0,1,0,0
3,"-117,613897","33,663005",0.80,0.40,0.58,0.06,1.00,06059,0.15,"472,384.00",...,medium,0,1,0,0,0,1,0,1,0
4,"-118,393309","34,227422",0.20,0.00,0.12,0.07,0.00,06037,0.55,"58,693.00",...,small,1,0,0,1,0,0,1,0,0


In [14]:
s_train, s_validate, s_test = prepare.split(scaled_df)

In [15]:
pd.set_option('display.max_columns', None)

In [16]:
cols = ['square_feet', 'lot_size', 'has_pool', 'age', 'county_Los Angeles', 'county_Orange', 'bedrooms', 'bathrooms']

X_train = s_train[cols]
y_train = s_train.assessed_value

X_validate = s_validate[cols]
y_validate = s_validate.assessed_value

X_test = s_test[cols]
y_test = s_test.assessed_value

In [17]:
train_predictions = pd.DataFrame({
    'actual': s_train.assessed_value
}) 
validate_predictions = pd.DataFrame({
    'actual': s_validate.assessed_value
}) 

In [18]:
final_train_predictions = pd.DataFrame({
    'actual': s_train.assessed_value
}) 
final_validate_predictions = pd.DataFrame({
    'actual': s_validate.assessed_value
}) 
final_test_predictions = pd.DataFrame({
    'actual': s_test.assessed_value
})

## Multiple  Regression + RFE

In [19]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
k = 6

# 1. Transform our X
rfe = RFE(lm, n_features_to_select=6)
rfe.fit(X_train, y_train)
print('selected top 8 features:', X_train.columns[rfe.support_])
X_train_rfe = rfe.transform(X_train)

selected top 8 features: Index(['square_feet', 'lot_size', 'has_pool', 'county_Los Angeles', 'bedrooms',
       'bathrooms'],
      dtype='object')


### Looping through k

In [20]:
for k in range(2,8):
    lm = LinearRegression()
    # 1. Transform our X
    rfe = RFE(lm, n_features_to_select=k)
    rfe.fit(X_train, y_train)
    # 2. Use the transformed x in our model
    X_train_rfe = rfe.transform(X_train)
    X_validate_rfe = rfe.transform(X_validate)
    lm.fit(X_train_rfe, y_train)
    
    # Make predictions
    train_predictions['multiple_rfe_k=', k] = lm.predict(X_train_rfe)
    validate_predictions['multiple_rfe_k=', k] = lm.predict(X_validate_rfe)
    k+=1

In [21]:
train_predictions.head()

,actual,"(multiple_rfe_k=, 2)","(multiple_rfe_k=, 3)","(multiple_rfe_k=, 4)","(multiple_rfe_k=, 5)","(multiple_rfe_k=, 6)","(multiple_rfe_k=, 7)"
36490,"702,091.00","365,446.15","336,583.84","331,268.49","307,288.35","333,329.05","332,132.09"
19337,"442,099.00","362,022.82","367,326.64","366,844.52","407,063.44","402,521.84","402,606.77"
10447,"76,613.00","337,243.53","338,393.10","341,151.05","320,630.55","316,125.78","315,559.81"
42559,"802,437.00","468,703.54","456,971.93","465,270.27","495,540.45","488,181.68","483,442.41"
34091,"188,011.00","635,997.56","585,372.02","587,489.63","550,727.63","565,314.42","568,408.62"


In [22]:
train_predictions['baseline'] = y_train.mean()

In [23]:
def calculate_rmse(y_predicted):
    return mean_squared_error(train_predictions.actual, y_predicted, squared = False)

train_predictions.apply(calculate_rmse).sort_values()

actual                       0.00
(multiple_rfe_k=, 7)   207,896.69
(multiple_rfe_k=, 6)   207,913.39
(multiple_rfe_k=, 5)   208,248.34
(multiple_rfe_k=, 4)   210,238.38
(multiple_rfe_k=, 3)   210,605.53
(multiple_rfe_k=, 2)   211,872.25
baseline               237,872.07
dtype: float64

In [24]:
validate_predictions['baseline'] = y_validate.mean()

In [25]:
validate_predictions.head()


,actual,"(multiple_rfe_k=, 2)","(multiple_rfe_k=, 3)","(multiple_rfe_k=, 4)","(multiple_rfe_k=, 5)","(multiple_rfe_k=, 6)","(multiple_rfe_k=, 7)",baseline
50904,"170,700.00","424,588.29","405,834.97","392,597.52","366,442.40","359,170.19","354,588.36","377,092.24"
22280,"395,742.00","501,825.12","530,898.33","538,752.58","571,359.34","595,129.29","593,258.35","377,092.24"
16371,"222,815.00","226,145.81","243,074.86","238,132.12","225,899.06","225,193.42","225,776.54","377,092.24"
33772,"816,260.00","404,804.07","417,595.86","438,151.75","413,479.96","406,814.12","411,092.34","377,092.24"
28672,"478,000.00","487,522.62","444,643.04","472,608.76","437,511.13","427,483.55","432,266.64","377,092.24"


In [26]:
def calculate_rmse(y_predicted):
    return mean_squared_error(validate_predictions.actual, y_predicted, squared = False)

validate_predictions.apply(calculate_rmse).sort_values()

actual                       0.00
(multiple_rfe_k=, 7)   205,507.56
(multiple_rfe_k=, 6)   205,561.22
(multiple_rfe_k=, 5)   205,988.21
(multiple_rfe_k=, 4)   207,997.73
(multiple_rfe_k=, 3)   208,212.92
(multiple_rfe_k=, 2)   209,451.26
baseline               235,949.69
dtype: float64

In [ ]:
# k=7 is the best

228490-227162

**K=7 has the lowest rmse, highest performance**

In [ ]:
# Adding k=7 to final table
lm = LinearRegression()
 # 1. Transform our X
    
rfe = RFE(lm, n_features_to_select=7)
rfe.fit(X_train, y_train)
# 2. Use the transformed x in our model
X_train_rfe = rfe.transform(X_train)
X_validate_rfe = rfe.transform(X_validate)
X_test_rfe = rfe.transform(X_test)
lm.fit(X_train_rfe, y_train)
    
# Make predictions
final_train_predictions['multiple_rfe_k=7'] = lm.predict(X_train_rfe)
final_validate_predictions['multiple_rfe_k=7'] = lm.predict(X_validate_rfe)
final_test_predictions['multiple_rfe_k=7']=lm.predict(X_test_rfe)

## Polynomial Features

In [27]:
train_pred = pd.DataFrame({
    'actual': s_train.assessed_value
}) 
validate_pred = pd.DataFrame({
    'actual': s_validate.assessed_value
}) 

In [28]:
from sklearn.preprocessing import PolynomialFeatures

# 1. Generate Polynomial Features, k=2
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)

In [29]:
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
train_pred['baseline'] = y_train.mean()
validate_pred['baseline'] = y_validate.mean()
train_pred['polynomial degree 2'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 2'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2
36490,"702,091.00","380,065.99","330,259.81"
19337,"442,099.00","380,065.99","388,463.44"
10447,"76,613.00","380,065.99","304,541.25"
42559,"802,437.00","380,065.99","663,056.97"
34091,"188,011.00","380,065.99","558,063.25"


In [30]:
validate_pred.head()

,actual,baseline,polynomial degree 2
50904,"170,700.00","377,092.24","312,777.74"
22280,"395,742.00","377,092.24","652,268.35"
16371,"222,815.00","377,092.24","258,911.62"
33772,"816,260.00","377,092.24","431,382.56"
28672,"478,000.00","377,092.24","457,754.49"


In [31]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 2'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 2'], squared = False)

In [32]:
train_rmse_b = mean_squared_error(train_pred.actual,train_pred['baseline'], squared = False)
validate_rmse_b = mean_squared_error(validate_pred.actual,validate_pred['baseline'], squared = False)

In [33]:
train_rmse, validate_rmse

(202295.3609893082, 199981.93217925238)

In [34]:
validate_rmse-train_rmse

-2313.428810055804

In [35]:
train_rmse_b, validate_rmse_b

(237872.06885584313, 235949.6867801346)

In [36]:
# Interaction terms only

poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)
lm = LinearRegression()
lm.fit(X_train_poly, y_train)

X_validate_poly = poly.transform(X_validate)
train_pred['polynomial degree 2 only interaction'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 2 only interaction'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2,polynomial degree 2 only interaction
36490,"702,091.00","380,065.99","330,259.81","330,661.04"
19337,"442,099.00","380,065.99","388,463.44","386,507.93"
10447,"76,613.00","380,065.99","304,541.25","303,762.97"
42559,"802,437.00","380,065.99","663,056.97","666,817.90"
34091,"188,011.00","380,065.99","558,063.25","565,992.28"


In [37]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 2 only interaction'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 2 only interaction'], squared = False)
train_rmse, validate_rmse

(202395.46231037687, 200172.90419428024)

In [38]:
validate_rmse-train_rmse

-2222.558116096625

In [39]:
# k=3
poly = PolynomialFeatures(degree=3, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)

In [40]:
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
train_pred['baseline'] = y_train.mean()
validate_pred['baseline'] = y_validate.mean()
train_pred['polynomial degree 3'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 3'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2,polynomial degree 2 only interaction,polynomial degree 3
36490,"702,091.00","380,065.99","330,259.81","330,661.04","324,717.82"
19337,"442,099.00","380,065.99","388,463.44","386,507.93","366,797.48"
10447,"76,613.00","380,065.99","304,541.25","303,762.97","309,260.45"
42559,"802,437.00","380,065.99","663,056.97","666,817.90","785,175.52"
34091,"188,011.00","380,065.99","558,063.25","565,992.28","592,092.20"


In [41]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 3'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 3'], squared = False)
train_rmse, validate_rmse

(200429.34706541212, 199529.26893855724)

In [42]:
# k=4
poly = PolynomialFeatures(degree=4, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)

In [43]:
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
train_pred['baseline'] = y_train.mean()
validate_pred['baseline'] = y_validate.mean()
train_pred['polynomial degree 4'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 4'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2,polynomial degree 2 only interaction,polynomial degree 3,polynomial degree 4
36490,"702,091.00","380,065.99","330,259.81","330,661.04","324,717.82","321,177.35"
19337,"442,099.00","380,065.99","388,463.44","386,507.93","366,797.48","361,572.64"
10447,"76,613.00","380,065.99","304,541.25","303,762.97","309,260.45","300,658.64"
42559,"802,437.00","380,065.99","663,056.97","666,817.90","785,175.52","760,988.45"
34091,"188,011.00","380,065.99","558,063.25","565,992.28","592,092.20","575,510.96"


In [44]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 4'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 4'], squared = False)
train_rmse, validate_rmse

(198875.59208328978, 200124.43203787412)

In [45]:
# k=5
poly = PolynomialFeatures(degree=5, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)

In [46]:
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
train_pred['baseline'] = y_train.mean()
validate_pred['baseline'] = y_validate.mean()
train_pred['polynomial degree 5'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 5'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2,polynomial degree 2 only interaction,polynomial degree 3,polynomial degree 4,polynomial degree 5
36490,"702,091.00","380,065.99","330,259.81","330,661.04","324,717.82","321,177.35","307,018.44"
19337,"442,099.00","380,065.99","388,463.44","386,507.93","366,797.48","361,572.64","364,794.92"
10447,"76,613.00","380,065.99","304,541.25","303,762.97","309,260.45","300,658.64","305,885.55"
42559,"802,437.00","380,065.99","663,056.97","666,817.90","785,175.52","760,988.45","744,694.18"
34091,"188,011.00","380,065.99","558,063.25","565,992.28","592,092.20","575,510.96","626,801.99"


In [47]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 4'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 4'], squared = False)
train_rmse, validate_rmse

(198875.59208328978, 200124.43203787412)

**K=3 has the lowest rmse**

In [ ]:
# Adding k=3 to final table
poly = PolynomialFeatures(degree=3, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
X_test_poly = poly.transform(X_test)
final_train_predictions['baseline'] = y_train.mean()
final_validate_predictions['baseline'] = y_validate.mean()
final_test_predictions['baseline'] = y_test.mean()
final_train_predictions['polynomial degree 3'] = lm.predict(X_train_poly)
final_validate_predictions['polynomial degree 3'] = lm.predict(X_validate_poly)
final_test_predictions['polynomial degree 3'] = lm.predict(X_test_poly)

## Lasso-Lars

In [48]:
from sklearn.linear_model import LassoLars
# create the model object
lars = LassoLars(alpha=0)

# fit the model to our training data
lars.fit(X_train, y_train)

# predict validate
X_train_pred_lars = lars.predict(X_train)

pd.Series(lars.coef_, index=X_train.columns).sort_values()

bedrooms             -203,239.61
lot_size             -154,118.26
county_Los Angeles    -64,778.51
county_Orange            -286.41
has_pool               34,416.29
age                    40,127.32
bathrooms             160,689.93
square_feet           770,247.49
dtype: float64

In [49]:
# create the model object
lars = LassoLars(alpha=1)

# fit the model to our training data
lars.fit(X_train, y_train)

# predict validate
X_train_pred_lars = lars.predict(X_train)
X_validate_pred_lars = lars.predict(X_validate)
# Add lassolars predictions to our predictions DataFrame
train_pred['lasso_lars'] = X_train_pred_lars
validate_pred['lasso_lars'] = X_validate_pred_lars

In [50]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['lasso_lars'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['lasso_lars'], squared = False)
train_rmse, validate_rmse

(208110.17015086598, 205840.74432369764)

In [51]:
validate_rmse-train_rmse

-2269.4258271683357

In [ ]:
# Adding to final table
# create the model object
lars = LassoLars(alpha=1)

# fit the model to our training data
lars.fit(X_train, y_train)

# predict validate
X_train_pred_lars = lars.predict(X_train)
X_validate_pred_lars = lars.predict(X_validate)
X_test_pred_lars = lars.predict(X_test)
# Add lassolars predictions to our predictions DataFrame
final_train_predictions['lasso_lars'] = X_train_pred_lars
final_validate_predictions['lasso_lars'] = X_validate_pred_lars
final_test_predictions['lasso_lars'] = X_test_pred_lars

## Generalized Linear Model

In [52]:
from sklearn.linear_model import TweedieRegressor

# create the model object
glm = TweedieRegressor(power=1, alpha=0)

# fit the model to our training data
glm.fit(X_train, y_train)

# predict train
X_train_predict_glm = glm.predict(X_train)
X_validate_predict_glm = glm.predict(X_validate)
# Add lassolars predictions to our predictions DataFrame
train_pred['glm'] = X_train_predict_glm
validate_pred['glm'] = X_validate_predict_glm

In [53]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['glm'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['glm'], squared = False)
train_rmse, validate_rmse

(209139.56707843233, 206658.25762675548)

In [54]:
validate_rmse-train_rmse

-2481.309451676847

In [55]:
# Adding to final table

glm = TweedieRegressor(power=1, alpha=0)

# fit the model to our training data
glm.fit(X_train, y_train)

# predict train
X_train_predict_glm = glm.predict(X_train)
X_validate_predict_glm = glm.predict(X_validate)
X_test_predict_glm = glm.predict(X_test)
# Add lassolars predictions to our predictions DataFrame
final_train_predictions['glm'] = X_train_predict_glm
final_validate_predictions['glm'] = X_validate_predict_glm
final_test_predictions['glm'] = X_test_predict_glm

## Evaluation

In [ ]:
def calculate_rmse(y_predicted):
    return mean_squared_error(final_train_predictions.actual, y_predicted, squared = False)

final_train_predictions.apply(calculate_rmse).sort_values()

In [ ]:
def calculate_rmse(y_predicted):
    return mean_squared_error(final_validate_predictions.actual, y_predicted, squared = False)

final_validate_predictions.apply(calculate_rmse).sort_values()

In [ ]:
def calculate_rmse(y_predicted):
    return mean_squared_error(final_test_predictions.actual, y_predicted, squared = False)

final_test_predictions.apply(calculate_rmse).sort_values()

In [ ]:
(277387.85-225323.49)/277387.85